In [4]:
# Hyperparameter tuning and optimization
import optuna
from ray.tune.search.optuna import OptunaSearch
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback

# PyTorch Lightning and callbacks
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, Callback, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

# Metrics
from torchmetrics import MeanAbsoluteError, MeanAbsolutePercentageError, MetricCollection

# Darts (Time series forecasting)
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import DLinearModel, LightGBMModel, BlockRNNModel, TiDEModel

# Data handling and preprocessing
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorboard

# System utilities
import os

Load Data / Spilt Data

In [5]:
# 步骤1: 加载CSV文件
df = pd.read_csv('../DataSet/EDvisitfileC.csv', encoding='ISO-8859-1')

# 确保'date'列是DateTime类型
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# 分割数据集为训练集、验证集和测试集（假设您已经根据时间排序）
train_end = 3237            #L: 3362, T:3372, Ka:3208, Ke:3274, Y:2557, C: 3237
val_end = 3602              #L: 3727, T:3737, Ka:3573, Ke:3639, Y:2822, C: 3602

# Split the DataFrame
train_df = df.iloc[:train_end]
val_df = df.iloc[train_end:val_end]
test_df = df.iloc[val_end:]

# 步骤2: 使用MinMaxScaler缩放数据
# 定义并拟合scaler
scaler = MinMaxScaler()
scaler.fit(train_df[['No']])  # 只用训练数据拟合scaler

# 缩放训练集和验证集
train_df.loc[:, 'No_scaled'] = scaler.transform(train_df[['No']])
val_df.loc[:, 'No_scaled'] = scaler.transform(val_df[['No']])
test_df.loc[:, 'No_scaled'] = scaler.transform(test_df[['No']])  # 用相同的scaler转换测试集以避免数据泄露

# 转换为TimeSeries对象
train_series = TimeSeries.from_dataframe(train_df, value_cols='No_scaled')
val_series = TimeSeries.from_dataframe(val_df, value_cols='No_scaled')
test_series = TimeSeries.from_dataframe(test_df, value_cols='No_scaled')

# 原始数据转换为TimeSeries对象，如果需要
train_series_origin = TimeSeries.from_dataframe(train_df, value_cols='No')
val_series_origin = TimeSeries.from_dataframe(val_df, value_cols='No')
test_series_origin = TimeSeries.from_dataframe(test_df, value_cols='No')

# 选择需要的列创建多变量时间序列(都是one hot coding)
columns = ['Dayoff', 'Mon', 'Tue', 'Wed', 'Thr', 'Fri', 'Sat', 'Sun', 'Dayscaled', 'NewYear', '3Lock', 
           'Outbreak','COVID19', 'Jan',	'Feb', 'Mar', 'Apr', 'Mar', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec'
          ]
df_multivariate = df[columns]

# 将DataFrame转换为多变量时间序列
ED_covariates = TimeSeries.from_dataframe(df_multivariate)

C:\Users\ian11\AppData\Local\Temp\ipykernel_5416\4252752960.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.loc[:, 'No_scaled'] = scaler.transform(train_df[['No']])
C:\Users\ian11\AppData\Local\Temp\ipykernel_5416\4252752960.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df.loc[:, 'No_scaled'] = scaler.transform(val_df[['No']])
C:\Users\ian11\AppData\Local\Temp\ipykernel_5416\4252752960.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

Hypertuning

In [6]:
class LossLoggingCallback(Callback):
    def __init__(self):
        super().__init__()
        self.val_losses = []  # To store validation losses
        self.train_losses = []  # To store training losses

    def on_validation_epoch_end(self, trainer, pl_module):
        val_loss = trainer.callback_metrics["val_loss"].item()
        self.val_losses.append(val_loss)
        print(f"Epoch {trainer.current_epoch}: val_loss={val_loss}")
        # Updated report call
        train.report({"loss": val_loss})  # Report the validation loss to Ray Train

    def on_train_epoch_end(self, trainer, pl_module, unused=None):
        if "train_loss" in trainer.callback_metrics:
            train_loss = trainer.callback_metrics["train_loss"].item()
            self.train_losses.append(train_loss)
            print(f"Epoch {trainer.current_epoch}: train_loss={train_loss}")
loss_logging_callback = LossLoggingCallback()
    

In [7]:
# Create the model using model_args from Ray Tune
def train_model(model_args, callbacks, train, val):
    torch_metrics = MetricCollection([MeanAbsolutePercentageError(), MeanAbsoluteError()])
    
    # Customize the ModelCheckpoint callback
    model_checkpoint_callback = ModelCheckpoint(
        dirpath="checkpoints",
        filename="{epoch}-{val_loss:.2f}",
        every_n_epochs=5,
    )
    
    model = DLinearModel(
        input_chunk_length=60,
        output_chunk_length=7,
        pl_trainer_kwargs={"callbacks": callbacks, "enable_progress_bar": False},
        log_tensorboard=True,
        **model_args)

    model.fit(
    series=[train_series],
    past_covariates=[ED_covariates],
    val_series=[val_series],
    val_past_covariates=[ED_covariates]
    )

In [8]:
# set up ray tune callback
config = {
    'kernel_size': tune.randint(5, 100),
    'lr_scheduler_kwargs': tune.uniform(0, 0.01),
}

# earlystopping
my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=5,
    min_delta=0.001,
    mode='min',
)


tune_callback = TuneReportCheckpointCallback(
    {
        "loss": "val_loss",
    },
    on="validation_end",
)



# define the hyperparameter space

reporter = CLIReporter(
    parameter_columns=list(config.keys()),
    metric_columns=["loss", "MAPE", "training_iteration"],
)

optuna_search = OptunaSearch(metric="loss", mode="min")

In [9]:
# Run Ray Tune, optimize hyperparameters by minimizing the MAPE on the validation set
num_samples = 30

scheduler = ASHAScheduler(max_t=1000, grace_period=5, reduction_factor=2)

train_fn_with_parameters = tune.with_parameters(
    train_model, callbacks=[my_stopper, tune_callback], train=train_series, val=val_series,
)

analysis = tune.run(
    train_fn_with_parameters,
    #resources_per_trial=resources_per_trial,
    metric="loss",  # any value in TuneReportCallback.
    mode="min",
    config=config,
    num_samples=num_samples,
    search_alg=optuna_search,
    scheduler=scheduler,
    progress_reporter=reporter,
    trial_dirname_creator=lambda trial: str(trial),
    name="tune_darts",
)

print("Best hyperparameters found were: ", analysis.best_config)

2024-09-23 21:42:51,843	INFO worker.py:1752 -- Started a local Ray instance.
2024-09-23 21:42:54,553	INFO tune.py:263 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2024-09-23 21:42:54,556	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
[I 2024-09-23 21:42:54,580] A new study created in memory with name: optuna


== Status ==
Current time: 2024-09-23 21:42:54 (running for 00:00:00.28)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 640.000: None | Iter 320.000: None | Iter 160.000: None | Iter 80.000: None | Iter 40.000: None | Iter 20.000: None | Iter 10.000: None | Iter 5.000: None
Logical resource usage: 1.0/16 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: C:/Users/ian11/AppData/Local/Temp/ray/session_2024-09-23_21-42-49_373180_5416/artifacts/2024-09-23_21-42-54/tune_darts/driver_artifacts
Number of trials: 1/30 (1 PENDING)
+----------------------+----------+-------+---------------+-----------------------+
| Trial name           | status   | loc   |   kernel_size |   lr_scheduler_kwargs |
|----------------------+----------+-------+---------------+-----------------------|
| train_model_a8f6ff1a | PENDING  |       |            33 |            0.00126183 |
+----------------------+----------+-------+---------------+-----------------------+




(raylet) Traceback (most recent call last):
(raylet)   File "C:\Users\ian11\anaconda3\envs\EDforecast\lib\site-packages\ray\_private\workers\setup_worker.py", line 33, in <module>
(raylet)     runtime_env_context.exec_worker(remaining_args, Language.Value(args.language))
(raylet)   File "C:\Users\ian11\anaconda3\envs\EDforecast\lib\site-packages\ray\_private\runtime_env\context.py", line 86, in exec_worker
(raylet)     subprocess.Popen(cmd, shell=True).wait()
(raylet)   File "C:\Users\ian11\anaconda3\envs\EDforecast\lib\subprocess.py", line 858, in __init__
(raylet)     self._execute_child(args, executable, preexec_fn, close_fds,
(raylet)   File "C:\Users\ian11\anaconda3\envs\EDforecast\lib\subprocess.py", line 1327, in _execute_child
(raylet)     hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
(raylet) PermissionError: [WinError 5] �s���Q�ڡC


== Status ==
Current time: 2024-09-23 21:43:00 (running for 00:00:05.34)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 640.000: None | Iter 320.000: None | Iter 160.000: None | Iter 80.000: None | Iter 40.000: None | Iter 20.000: None | Iter 10.000: None | Iter 5.000: None
Logical resource usage: 1.0/16 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: C:/Users/ian11/AppData/Local/Temp/ray/session_2024-09-23_21-42-49_373180_5416/artifacts/2024-09-23_21-42-54/tune_darts/driver_artifacts
Number of trials: 1/30 (1 PENDING)
+----------------------+----------+-------+---------------+-----------------------+
| Trial name           | status   | loc   |   kernel_size |   lr_scheduler_kwargs |
|----------------------+----------+-------+---------------+-----------------------|
| train_model_a8f6ff1a | PENDING  |       |            33 |            0.00126183 |
+----------------------+----------+-------+---------------+-----------------------+


== Status ==
Current time: 2024-09-23 2

2024-09-23 21:43:18,122	WARNING tune.py:229 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-09-23 21:43:18,128	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-09-23 21:43:18,132	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to 'C:/Users/ian11/ray_results/tune_darts' in 0.0080s.


== Status ==
Current time: 2024-09-23 21:43:18 (running for 00:00:23.45)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 640.000: None | Iter 320.000: None | Iter 160.000: None | Iter 80.000: None | Iter 40.000: None | Iter 20.000: None | Iter 10.000: None | Iter 5.000: None
Logical resource usage: 1.0/16 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: C:/Users/ian11/AppData/Local/Temp/ray/session_2024-09-23_21-42-49_373180_5416/artifacts/2024-09-23_21-42-54/tune_darts/driver_artifacts
Number of trials: 1/30 (1 PENDING)
+----------------------+----------+-------+---------------+-----------------------+
| Trial name           | status   | loc   |   kernel_size |   lr_scheduler_kwargs |
|----------------------+----------+-------+---------------+-----------------------|
| train_model_a8f6ff1a | PENDING  |       |            33 |            0.00126183 |
+----------------------+----------+-------+---------------+-----------------------+




2024-09-23 21:43:28,209	INFO tune.py:1048 -- Total run time: 33.65 seconds (23.44 seconds for the tuning loop).
2024-09-23 21:43:28,211	WARNING tune.py:1063 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2024-09-23 21:43:28,218	WARNING experiment_analysis.py:190 -- Failed to fetch metrics for 1 trial(s):
- train_model_a8f6ff1a: FileNotFoundError('Could not fetch metrics for train_model_a8f6ff1a: both result.json and progress.csv were not found at C:/Users/ian11/ray_results/tune_darts/train_model_a8f6ff1a')


Best hyperparameters found were:  {'kernel_size': 33, 'lr_scheduler_kwargs': 0.0012618337846478911}


In [10]:
import plotly.express as px
pd.DataFrame.iteritems = pd.DataFrame.items

df = analysis.results_df

# 假設 df 是你的 DataFrame
fig = px.parallel_coordinates(df, 
                              dimensions=['config/kernel_size', 'config/lr_scheduler_kwargs', 'loss'],
                              color='loss',
                              labels={"config/kernel_size": "Kernel Size",
                                      "config/lr_scheduler_kwargs": "Learning Rate",
                                      "loss": "Loss"},
                              color_continuous_scale=px.colors.diverging.Tealrose,  # 色彩範圍
                              #color_continuous_midpoint=0.004
                             )  # 中間點，根據數據適當調整

# 設定每個維度的範圍
fig.update_traces(dimensions=[
    dict(range=[5, 100], label='Kernel Size', values=df['config/kernel_size']),
    dict(range=[0, 0.01], label='Learning Rate', values=df['config/lr_scheduler_kwargs']),
    dict(range=[min(df['loss']), max(df['loss'])], label='Loss', values=df['loss'])
])

fig.show()

ModuleNotFoundError: No module named 'plotly'

(raylet) [2024-09-23 21:43:54,925 E 29176 25916] (raylet.exe) worker_pool.cc:550: Some workers of the worker process(25964) have not registered within the timeout. The process is dead, probably it crashed during start.


In [ ]:
df.to_csv('C:\\Users\\ian11\\EDtimeseriesForecast\\EDtimeseriesForecast\\Result\\DLinear\\Chiayi\\Hypertuning.csv')